# Setting up

In [1]:
!apt install swig cmake




cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 68 not upgraded.
Need to get 1116 kB of archives.
After this operation, 5542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5632 B]
Fetched 1116 kB in 2s (566 kB/s)

78Selecting previously unselected package swig4.0.
(Reading database ... 122996 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######....................................................] 8Unpacking swig4.0 (4.0.2-1ubuntu1) ...
7Progress: [ 22%

In [2]:
!pip install stable-baselines3[extra]
!pip install huggingface_sb3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.5 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446667 sha256=5132de4fb967e4e0793a3bab9a8413e38d6ce46abb5c67e8dbb04c40c7ca2399
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.3.0
    Uninstalling Shimmy-1.3.0:
      Successfully uninstalled Shimmy-1.3.0
ERROR: pip's dependency resolver does not currently take into account all th

In [3]:
!pip install optuna

# Create a virtual display

In [4]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay




E: Unable to locate package python-opengl



ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 68 not upgraded.



xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.11).
0 upgraded, 0 newly installed, 0 to remove and 68 not upgraded.


In [5]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Imports

In [6]:
import gymnasium as gym
import numpy as np
import os
from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

# Hyperparameter tuning

In [7]:
print(os.cpu_count())

4


### Config

In [8]:
# Pendulum truncates at 200 time steps per episode
# So 30,000 time steps means at least 150 episodes
env_id = "Pendulum-v1"
n_trials = 50 # Maximum number of trials
n_jobs = os.cpu_count() # Number of jobs to run in parallel
n_startup_trials = 5 # Stop random sampling after n_startup_trials
n_evaluations = 3 # Number of evaluations during the training
n_timesteps = 30_000 # Maximum number of timesteps
eval_freq = int(n_timesteps / n_evaluations)
n_eval_envs = 5
n_eval_episodes = 10
timeout = 3 * 3600 # 3 hours

default_hyperparams = {
    "policy": "MlpPolicy",
    "env": env_id,
    "buffer_size": 100_000,
}

### Define the search space

In [9]:
from typing import Any, Dict

def sample_sac_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Sampler for DQN hyperparameters.
    Args:
        trial: Optuna trial object.
    Returns:
        Dictionary of sampled hyperparameters for the given trial.
    """

    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        "gamma": trial.suggest_float("gamma", 0.9, 0.999),
        "tau": trial.suggest_float("tau", 1e-4, 1e-2, log=True),
        "train_freq": trial.suggest_categorical("train_freq", [1, 4, 8]),
        "gradient_steps": trial.suggest_categorical("gradient_steps", [1, 4, 8]),
        "policy_kwargs": {
            "net_arch": trial.suggest_categorical("net_arch", [[64, 64], [128, 128], [256, 256]]),
        },
    }

### Define the objective function
First we define a custom callback to report the results of periodic evaluations to Optuna.

In [10]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    Callback used for evaluating and reporting a trial.

    :param eval_env: Evaluation environment
    :param trial: Optuna trial object
    :param n_eval_episodes: Number of evaluation episodes
    :param eval_freq: Evaluate the agent every ``eval_freq`` call of the callback.
    :param deterministic: Whether the evaluation should
        use a stochastic or deterministic policy.
    :param verbose:
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10_000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

Then we define the objective function that is in charge of sampling hyperparameters, creating the model and then returning the result to Optuna

In [11]:
def objective(trial: optuna.Trial) -> float:
    """
    Objective function used by Optuna to evaluate one configuration (i.e., one set of hyperparameters).

    Given a trial object, it will sample hyperparameters, evaluate it and report the result
    (mean episodic reward after training)

    Args:
        trial: Optuna trial object
    Returns:
        Mean episodic reward after training
    """

    kwargs = default_hyperparams.copy()
    # Sample parameters
    kwargs.update(sample_sac_params(trial))

    # Create the RL model
    model = SAC(**kwargs)

    # Create evaluation environment
    eval_env = make_vec_env(env_id, n_envs=n_eval_envs)

    # Create the callback that will periodically evaluate
    eval_callback = TrialEvalCallback(
        eval_env=eval_env,
        trial=trial,
        n_eval_episodes=n_eval_episodes,
        eval_freq=eval_freq,
        verbose=1,
    )

    nan_encountered = False
    try:
        model.learn(n_timesteps, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

### The optimization loop

In [12]:
sampler = TPESampler(n_startup_trials=n_startup_trials)
# Do not prune before 1/3 of the max budget is used
pruner = MedianPruner(
    n_startup_trials=n_startup_trials, n_warmup_steps=(n_timesteps // 3)
)
# Create the study and start the hyperparameter optimization
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, timeout=timeout)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Write report
study.trials_dataframe().to_csv("study_results_sac_pendulum.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

[I 2024-11-21 12:35:03,695] A new study created in memory with name: no-name-c7c24ec3-10d9-4561-b622-9c35ceb0b6d6
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64, 64] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 256] which is of type list.
  warnings.warn(message)


Eval num_timesteps=10000, episode_reward=-1675.18 +/- 177.37
Episode length: 200.00 +/- 0.00
New best mean reward!
Eval num_timesteps=20000, episode_reward=-1359.56 +/- 89.40
Episode length: 200.00 +/- 0.00
New best mean reward!
Eval num_timesteps=10000, episode_reward=-799.37 +/- 146.53
Episode length: 200.00 +/- 0.00
New best mean reward!
Eval num_timesteps=30000, episode_reward=-1070.04 +/- 42.71
Episode length: 200.00 +/- 0.00
New best mean reward!


[I 2024-11-21 13:05:15,967] Trial 1 finished with value: -1070.0433832999997 and parameters: {'learning_rate': 4.1165675843494636e-05, 'gamma': 0.9376719896785453, 'tau': 0.00027985579791229393, 'train_freq': 8, 'gradient_steps': 8, 'net_arch': [256, 256]}. Best is trial 1 with value: -1070.0433832999997.
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64, 64] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128] which is of type list.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, 

Eval num_timesteps=10000, episode_reward=-143.96 +/- 70.17
Episode length: 200.00 +/- 0.00
New best mean reward!
Eval num_timesteps=20000, episode_reward=-169.86 +/- 99.62
Episode length: 200.00 +/- 0.00
New best mean reward!
Eval num_timesteps=10000, episode_reward=-288.26 +/- 404.13
Episode length: 200.00 +/- 0.00
New best mean reward!


[I 2024-11-21 13:58:28,066] Trial 2 finished with value: -161.48442110000002 and parameters: {'learning_rate': 3.657259260015599e-05, 'gamma': 0.927201947300053, 'tau': 0.000320323003838478, 'train_freq': 1, 'gradient_steps': 4, 'net_arch': [64, 64]}. Best is trial 2 with value: -161.48442110000002.


...

Eval num_timesteps=10000, episode_reward=-145.25 +/- 43.19
Episode length: 200.00 +/- 0.00
New best mean reward!


[I 2024-11-21 16:04:17,168] Trial 4 finished with value: -143.49746259999998 and parameters: {'learning_rate': 1.9076243201421694e-05, 'gamma': 0.9297866969132312, 'tau': 0.0005435995778071192, 'train_freq': 1, 'gradient_steps': 8, 'net_arch': [64, 64]}. Best is trial 4 with value: -143.49746259999998.


Eval num_timesteps=30000, episode_reward=-143.50 +/- 92.53
Episode length: 200.00 +/- 0.00
New best mean reward!
Eval num_timesteps=20000, episode_reward=-143.90 +/- 42.12
Episode length: 200.00 +/- 0.00
New best mean reward!


[I 2024-11-21 16:19:54,144] Trial 9 finished with value: -165.96314359999997 and parameters: {'learning_rate': 0.005947781241868968, 'gamma': 0.9870589052406724, 'tau': 0.0019063764765887019, 'train_freq': 1, 'gradient_steps': 8, 'net_arch': [64, 64]}. Best is trial 4 with value: -143.49746259999998.


Eval num_timesteps=30000, episode_reward=-165.96 +/- 76.19
Episode length: 200.00 +/- 0.00
Number of finished trials:  10
Best trial:
  Value: -143.49746259999998
  Params: 
    learning_rate: 1.9076243201421694e-05
    gamma: 0.9297866969132312
    tau: 0.0005435995778071192
    train_freq: 1
    gradient_steps: 8
    net_arch: [64, 64]


# Train the Final Agent

In [13]:

# Extract the net_arch param into policy_kwargs
best_params = default_hyperparams.copy()
best_params.update(trial.params)
net_arch = best_params.pop('net_arch')
best_params["policy_kwargs"] = { "net_arch": net_arch }
best_model = SAC(**best_params)
best_model.learn(total_timesteps=100_000)
model_name = "sac-pendulum-v1-optuna"
best_model.save(model_name)

# Evaluate the Agent

In [14]:
eval_env = Monitor(gym.make(env_id))
mean_reward, std_reward = evaluate_policy(best_model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean reward: {mean_reward: .2f}, +/- {std_reward}")

mean reward: -182.79, +/- 75.38501039054042


# Publish my trained model on the Huggingface Hub 🔥

In [ ]:
notebook_login()
!git config --global credential.helper store

In [16]:
from stable_baselines3.common.vec_env import DummyVecEnv

env_id = "Pendulum-v1"
model_architecture = "SAC"
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {username}/{repo_name}
repo_id = "wowthecoder/sac-pendulum-v1-optuna"
commit_msg = "Upload SAC Pendulum agent with tuned hyperparameters from Optuna"
# Create the eval env and set the render mode = "rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])

# package_to_hub(
#     model=best_model,
#     model_name=model_name,
#     model_architecture=model_architecture,
#     env_id=env_id,
#     eval_env = eval_env,
#     repo_id = repo_id,
#     commit_message=commit_msg,
# )